In [3]:
import os, glob
import cv2
import numpy as np

# =======================
# 경로
# =======================
ROOT = "car"
BEFORE_DIR = os.path.join(ROOT, "before")
AFTER_DIR  = os.path.join(ROOT, "after")
HOT_MASK_PATH = os.path.join(ROOT, "hotspot_mask.png")  # 없으면 자동 OFF

OUT_DIR = os.path.join(ROOT, "boxes_vis_roi")
os.makedirs(OUT_DIR, exist_ok=True)

EXTS = [".png", ".jpg", ".jpeg", ".PNG", ".JPG", ".JPEG"]

# =======================
# ROI (기준 해상도에서 잡고 -> 타겟 크기에 맞게 자동 스케일)
# =======================
REF_W, REF_H = 1920, 1080
ROI_REF = (100, 100, 1800, 950)  # (x1,y1,x2,y2)

# =======================
# 파라미터
# =======================
USE_ECC = True
ECC_ITERS = 60
ECC_EPS = 1e-5

BLUR = 0
PCTL = 98.0
ABS_THR = 5

LINE_LEN = 41
LINE_PCTL = 98.8
LINE_MIN = 3

BRIDGE_K = 9
BRIDGE_ITERS = 1

MIN_AREA = 3
MAX_BOXES = 30

MIN_ASPECT = 2.0
MAX_ASPECT = 400.0

MIN_FILL = 0.00003
MIN_SCORE = 2.0

MIN_CHANGED_PIXELS = 15
MIN_CHANGED_RATIO  = 0.00002
MAX_BOX_AREA_RATIO = 0.12

USE_EDGE_BAND = True
CANNY1, CANNY2 = 90, 240
EDGE_DILATE = 3

USE_HOTSPOT = True
HOTSPOT_DISABLE_IF_COVERAGE_GT = 0.20

# 출력 옵션
SHOW_DIFF_PANEL = False   # True면 before|after|diff 3분할, False면 before|after 2분할

# =======================
# 유틸
# =======================
def find_before(noext):
    for e in EXTS:
        p = os.path.join(BEFORE_DIR, noext + e)
        if os.path.exists(p):
            return p
    return None

def preprocess_gray(img):
    g = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return cv2.createCLAHE(2.0, (8, 8)).apply(g)

def scale_roi_to_target(w, h):
    x1, y1, x2, y2 = ROI_REF
    rx1 = int(x1 / REF_W * w)
    ry1 = int(y1 / REF_H * h)
    rx2 = int(x2 / REF_W * w)
    ry2 = int(y2 / REF_H * h)

    rx1 = max(0, min(w - 1, rx1))
    ry1 = max(0, min(h - 1, ry1))
    rx2 = max(rx1 + 1, min(w, rx2))
    ry2 = max(ry1 + 1, min(h, ry2))
    return (rx1, ry1, rx2, ry2)

def align_ecc_roi(ref, mov, roi):
    x1, y1, x2, y2 = roi

    # ✅ dtype를 위치인자가 아니라 키워드로!
    warp = np.eye(2, 3, dtype=np.float32)

    try:
        cv2.findTransformECC(
            ref[y1:y2, x1:x2],
            mov[y1:y2, x1:x2],
            warp,
            cv2.MOTION_EUCLIDEAN,
            (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, ECC_ITERS, ECC_EPS)
        )
        aligned = cv2.warpAffine(
            mov,
            warp,
            (ref.shape[1], ref.shape[0]),
            flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP
        )
        return aligned
    except cv2.error:
        return mov
    except Exception:
        return mov

def boxes_from_mask(mask, min_area):
    cs, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    out = []
    for c in cs:
        a = cv2.contourArea(c)
        if a < min_area:
            continue
        x, y, w, h = cv2.boundingRect(c)
        out.append((x, y, x + w, y + h, float(a)))
    return sorted(out, key=lambda x: x[4], reverse=True)

def filter_boxes(boxes, mask, gb, ga, roi):
    keep = []
    rx1, ry1, rx2, ry2 = roi
    roi_area = max(1, (rx2 - rx1) * (ry2 - ry1))

    H, W = mask.shape[:2]

    for x1, y1, x2, y2, a in boxes:
        x1 = max(0, min(W - 1, x1))
        x2 = max(1, min(W, x2))
        y1 = max(0, min(H - 1, y1))
        y2 = max(1, min(H, y2))
        if x2 <= x1 or y2 <= y1:
            continue

        w, h = x2 - x1, y2 - y1
        aspect = max(w / max(1, h), h / max(1, w))
        if not (MIN_ASPECT <= aspect <= MAX_ASPECT):
            continue

        if (w * h) > MAX_BOX_AREA_RATIO * roi_area:
            continue

        fill = float((mask[y1:y2, x1:x2] > 0).mean())
        if fill < MIN_FILL:
            continue

        score = float(np.percentile(
            cv2.absdiff(ga[y1:y2, x1:x2], gb[y1:y2, x1:x2]),
            95
        ))
        if score < MIN_SCORE:
            continue

        keep.append((x1, y1, x2, y2, a, fill, score, aspect))

    keep.sort(key=lambda x: (x[6], x[4]), reverse=True)
    return keep[:MAX_BOXES]

# =======================
# 마스크 생성
# =======================
def make_masks(bgr_b, bgr_a, hot, roi):
    h, w = hot.shape[:2]

    b = cv2.resize(bgr_b, (w, h), interpolation=cv2.INTER_AREA)
    a = cv2.resize(bgr_a, (w, h), interpolation=cv2.INTER_AREA)

    gb, ga = preprocess_gray(b), preprocess_gray(a)
    if USE_ECC:
        ga = align_ecc_roi(gb, ga, roi)

    diff = cv2.absdiff(ga, gb)

    if BLUR > 0:
        k = BLUR if (BLUR % 2 == 1) else (BLUR + 1)
        diff = cv2.GaussianBlur(diff, (k, k), 0)

    x1, y1, x2, y2 = roi
    thr = max(float(np.percentile(diff[y1:y2, x1:x2], PCTL)), ABS_THR)

    mask = (diff >= thr).astype(np.uint8) * 255

    # 라인 강조 (tophat + blackhat)
    L = (LINE_LEN | 1)
    k1 = cv2.getStructuringElement(cv2.MORPH_RECT, (L, 1))
    k2 = cv2.getStructuringElement(cv2.MORPH_RECT, (1, L))

    line = cv2.max(
        cv2.morphologyEx(diff, cv2.MORPH_TOPHAT, k1),
        cv2.morphologyEx(diff, cv2.MORPH_TOPHAT, k2)
    )
    line = cv2.max(line, cv2.morphologyEx(diff, cv2.MORPH_BLACKHAT, k1))
    line = cv2.max(line, cv2.morphologyEx(diff, cv2.MORPH_BLACKHAT, k2))

    t = float(np.percentile(line[y1:y2, x1:x2], LINE_PCTL))
    mask_line = ((line >= max(t, LINE_MIN)).astype(np.uint8) * 255)
    mask = cv2.bitwise_or(mask, mask_line)

    # hotspot 제거
    if USE_HOTSPOT and hot is not None:
        mask = cv2.bitwise_and(mask, cv2.bitwise_not(hot))

    # edge 제거
    if USE_EDGE_BAND:
        e = cv2.Canny(ga, CANNY1, CANNY2) | cv2.Canny(gb, CANNY1, CANNY2)
        e = cv2.dilate(e, np.ones((EDGE_DILATE, EDGE_DILATE), np.uint8), iterations=1)
        mask = cv2.bitwise_and(mask, cv2.bitwise_not(e))

    # 라인 연결
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, np.ones((1, BRIDGE_K), np.uint8), iterations=BRIDGE_ITERS)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, np.ones((BRIDGE_K, 1), np.uint8), iterations=BRIDGE_ITERS)

    # 변화 없음 컷
    changed = int((mask[y1:y2, x1:x2] > 0).sum())
    roi_area = int((y2 - y1) * (x2 - x1))
    ratio = changed / max(1, roi_area)
    if changed < MIN_CHANGED_PIXELS or ratio < MIN_CHANGED_RATIO:
        mask[:] = 0

    return mask, ga, gb, diff, thr

# =======================
# 메인
# =======================
def main():
    hot = np.zeros((1, 1), np.uint8)
    if USE_HOTSPOT and os.path.exists(HOT_MASK_PATH):
        tmp = cv2.imread(HOT_MASK_PATH, 0)
        if tmp is not None:
            hot = ((tmp > 0).astype(np.uint8) * 255)

    files = sum([glob.glob(os.path.join(AFTER_DIR, "*" + e)) for e in EXTS], [])
    files = sorted(files)
    if not files:
        print("after 파일 없음:", AFTER_DIR)
        return

    for af in files:
        name = os.path.splitext(os.path.basename(af))[0]
        bf = find_before(name)
        if not bf:
            continue

        after_color  = cv2.imread(af)
        before_color = cv2.imread(bf)
        if after_color is None or before_color is None:
            continue

        # hotspot이 더미면 after 크기로 맞춤
        if hot.shape == (1, 1):
            hot = np.zeros(after_color.shape[:2], np.uint8)

        # hotspot coverage가 너무 크면 OFF
        if USE_HOTSPOT and hot.size > 1:
            cov = float((hot > 0).mean())
            if cov > HOTSPOT_DISABLE_IF_COVERAGE_GT:
                print("⚠ hotspot too big -> OFF:", cov)
                hot[:] = 0

        Ht, Wt = hot.shape[:2]
        roi = scale_roi_to_target(Wt, Ht)

        mask, ga, gb, diff, thr = make_masks(before_color, after_color, hot, roi)
        boxes = filter_boxes(boxes_from_mask(mask, MIN_AREA), mask, gb, ga, roi)

        # ✅ 컬러로 출력 (좌표계 맞추기 위해 hot 크기로 resize)
        after_vis  = cv2.resize(after_color,  (Wt, Ht), interpolation=cv2.INTER_AREA)
        before_vis = cv2.resize(before_color, (Wt, Ht), interpolation=cv2.INTER_AREA)

        for x1, y1, x2, y2, _, _, _, _ in boxes:
            cv2.rectangle(after_vis, (x1, y1), (x2, y2), (0, 255, 0), 2)

        if SHOW_DIFF_PANEL:
            diff_vis = cv2.cvtColor(diff, cv2.COLOR_GRAY2BGR)
            panel = np.hstack([before_vis, after_vis, diff_vis])
            out_path = os.path.join(OUT_DIR, name + "_triple.jpg")
        else:
            panel = np.hstack([before_vis, after_vis])
            out_path = os.path.join(OUT_DIR, name + "_pair.jpg")

        cv2.imwrite(out_path, panel)

    print("완료:", OUT_DIR)

if __name__ == "__main__":
    main()


⚠ hotspot too big -> OFF: 1.0
완료: car\boxes_vis_roi
